In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

In [2]:
# reader = emcee.backends.HDFBackend("output/backend_20231103t1828.hdf")
# reader = emcee.backends.HDFBackend("output/backend_20231105t1259.hdf")
reader = emcee.backends.HDFBackend("../outputs/backend_20231112t1756.hdf")
print(reader.shape)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../outputs/backend_20231112t1756.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
tau = reader.get_autocorr_time(tol=1)

In [ ]:
chain = reader.get_chain()

In [ ]:
acc_rate = reader.accepted/chain.shape[0]

In [ ]:
print(acc_rate.mean())

In [ ]:
print(tau.mean())
print(np.median(tau))

In [ ]:
print(chain.shape)

In [ ]:
print(tau)

In [ ]:
plt.plot(chain[:, :, 10])

In [ ]:
flat_chain = reader.get_chain(discard=225, flat=True)
print(flat_chain.shape)

In [ ]:
labels = ["idrk", "W11", "W12", "W21", "W22", "W31", "W32", "W41", "W42", "W51", "W52", "W61", "W62"
          , "C3", "C4", "C5", "T3", "T4", "T5"]

In [ ]:
for i in range(flat_chain.shape[1]):
    fig = corner.corner(flat_chain[:, i:i+1], labels=labels[i])

In [ ]:
fig = corner.corner(flat_chain, labels=labels)